# Create STAC

## STAC Structure

```txt
Catalog: NatureScan
|-- Collection: RGB Orthomosaics
|   |-- Item: RGB Item 1
|   |   |-- Asset: Main (RGB Orthomosaic)
|   |   |-- Asset: Thumbnail
|   |-- Item: RGB Item 2
|   |   |-- Asset: Main (RGB Orthomosaic)
|   |   |-- Asset: Thumbnail
|-- Collection: MS Orthomosaics
|   |-- Item: RGB Item 1
|   |   |-- Asset: Main (MS Orthomosaic)
|   |   |-- Asset: Thumbnail
|   |-- Item: RGB Item 2
|   |   |-- Asset: Main (MS Orthomosaic)
|   |   |-- Asset: Thumbnail
|-- Collection: SfM Point Clouds (Eventually)
```

## STAC Spec

We are using the latest STAC 1.1. This includes the common metadata (some of which is previously in the bands and raster extension).

### Common Metadata

See [common metadata spec page](https://github.com/radiantearth/stac-spec/blob/master/commons/common-metadata.md)

- Basics
    - `title`, `description`: as spected
    - `keywords`, `roles`: TBC
- Date & Time
    - `datetime`: as expected. Though it is recommended for drones we use a start and end time
- Licensing
    - `license`: Exact details TBC. Should be added at a collection level
- Provider
    - provider object: Exact details TBC. Terra Luma def should be included.
- Instrument
    - `platform`: DJI Mavic 3M (item)
    - `instruments`: [DJI Mavic 3M RGB Camera] (item)
    - `mission`: NTAFIN0013 (item)
    - `gsd`: (item) if we can
- Bands
    - Can include some of this information at the collection level. But we should
      still include it at the asset level. Slight variations for previous eo and raster extensions
    - `name`: unique name, e.g. b1 or eed
    - `description`: For us likely same as name (or not needed)
- Data Values (can go at asset level or band level)
    - `nodata`: -32767.0 (asset)
    - `data_type`: float32 (asset)
    - `statistics`: min, max, mean, stddev (band)
    - `unit`: ?? should not be metre though

### Proj Extension 2.0

Most of this can come directly from GDAL raster info.

- `proj:code`: GDAL provides as proj:epsg
- `proj:wkt2`: GDAL provides
- `proj:projjson`: GDAL provides but it [needs reordering](https://github.com/stac-extensions/projection?tab=readme-ov-file#projtransform)
- `proj:shape`: GDAL provides
- `proj:transform`: GDAL provides

Seemingly that's all that's needed to comply with best practices. However, proj:geometry, proj:bbox 
and proj:centroid can also be derived fairly easily from other parts of GDAL info.

- `proj:geometry` - GeoJSON but in original CRS
- `proj:bbox` - BBox in original CRS
- `proj:centroid` - Centroid coordinates but in lat/lng

### EO Extension 2.0

These (expect maybe common_name and center wavelength) need to be provided. (i.e. they won't reliably be in the COG metadata).

- `eo:common_name` (band)
- `eo:center_wavelength` (band)
- `eo:full_width_half_max` (band)
- `eo:cloud_cover`: Might have from user metadata (estimated percentage) (asset)
- `eo:snow_cover`: N/A? (asset)

Previously included eo:bands but is not required anymore as bands are in common metadata. The presence of eo: fields indicate that a band is spectral.

Might need some info on what the values are.

### Raster Extension 2.0

- `raster:sampling`: area (asset)
- `raster:bits_per_sample`: not required as we are using a standard amount of bits per the data_type
- `raster:spatial_resolution`: (asset) Average spatial resolution (in meters) of the pixels in the band. How does this differ to GSD?
- `raster:scale`: Multiplicator factor of the pixel value to transform into the value (i.e. translate digital number to reflectance). I don't think we have
- `raster:offset`: Number to be added to the pixel value (after scaling) to transform into the value (i.e. translate digital number to reflectance). I don't think we have
- `raster:histogram`: (band) Histogram distribution information of the pixels values in the band. 
    - histogram object
    - count: # of buckets
    - min: min value or mean value of first bucket
    - max: max value or mean value of last bucket
    - buckets: number[] # of pixels per bucket

## Detailed Structure

```yaml
# Collection
title: NatureScan RGB Orthomosaics                                  # Hard Coded ✅
description: ...                                                    # Hard Coded ✅
license: TBC                                                        # Hard Coded 🚧
providers:                                                          # Hard Coded ✅🚧
    -   name: TerraLuma
        description: ...
        url: https://www.utas.edu.au/research/projects/terraluma
        roles:
            - licensor
            - producer
            - processor
            - host

# Item
title: ...                                                          # Derived from metadata (filename) ✅
datetime: ...                                                       # Derived from metadata (filename) ✅
mission: ...                                                        # Derived from metadata (filename) ✅
platform: DJI Mavic 3M                                              # Hard Coded ✅
instruments: [DJI Mavic 3M RGB Camera]                              # Hard Coded ✅
# -- proj extension --
proj:code: from GDAL info stac                                      # Derived from dataset ✅
proje:wkt2: from GDAL info stac                                     # Derived from dataset ✅
proj:projjson: from GDAL info stac                                  # Derived from dataset ✅
proj:shape: from GDAL info stac                                     # Derived from dataset ✅
proj:transform: from GDAL info geoTransform with reordering         # Derived from dataset ✅
proj:geometry: TBC from GDAL info corners                           # Derived from dataset 🚧
proj:bbox: TBC from GDAL info corners                               # Derived from dataset 🚧
proj:centroid: TBC form GDAL info wgs84 extent                      # Derived from dataset 🚧

# Main Asset
nodata: 255                                                         # Derived from dataset (expected) ✅
data_type: uint8                                                    # Derived from dataset (expected) ✅
unit: TBC                                                           # ? 🚧
# -- eo extension --
eo:cloud_cover: TBC if possible                                     # Derived from metadata (TBC) 🚧
# -- raster extension --
raster:sampling: area                                               # literal value ✅
raster:spatial_resolution: TBC if I can work out                    # Derived from dataset (TBC how) 🚧
# bands
bands:
    -   name: b1                                                    # Hard coded* ✅
        description: Red (R)                                        # Hard coded* ✅
        statistics:                                                 # Derived from dataset ✅
            minimum: ...
            maximum: ...
            mean: ...
            stddev: ...
        eo:common_name: red                                         # Hard coded* ✅
        eo:center_wavelength: ...                                   # Hard coded* ✅
        eo:full_width_half_max: ...                                 # Hard coded* ✅
        raster:historgram: ...                                      # Derived from dataset ✅
            count: ..
            min: ..
            max: ..
            buckets: ..
    -   ...
```

Comments explain where data comes from:
- hard coded - values are coded into the STAC pipeline. Could also be considered user provided. The `*` is marked for fields that could in theory be derived from the dataset but are instead provided for ease as the GDAL info stac `eo:bands`, `raster:bands` and `bands` fields are not 100% what we want.
- derived from dataset - values should be within required COG metadata (e.g. geospatial info)
- derived from metadata - values derived from metadata, be it from the COG, from the COG file name or other source



In [130]:
from pathlib import Path
from osgeo import gdal
import pystac

from datetime import datetime
from zoneinfo import ZoneInfo

In [171]:
input_dir = Path("/Volumes/Sammy/terra-luma/stac-input")
stac_out_dir = Path("/Volumes/Sammy/terra-luma/stac-output")
stac_out_dir.mkdir(exist_ok=True)
cogs = list(input_dir.rglob('*.cog.tif'))

In [172]:
from pydantic import BaseModel

class BandConfig(BaseModel):
    name: str
    description: str
    common_name: str
    center_wavelength: float | None = None
    full_width_half_max: float | None = None

class AssetConfig(BaseModel):
    title: str
    description: str | None = None
    nodata: int | float
    # Byte or Float32
    data_type: str
    bands: list[BandConfig]


RGB_ASSET_CONFIG = AssetConfig(
    title="RGB Orthmosaic Asset",
    nodata=255,
    data_type="Byte",
    bands=[
        BandConfig(name='b1', description='Red (R)', common_name='red'),
        BandConfig(name='b2', description='Green (G)', common_name='blue'),
        BandConfig(name='b3', description='Blue (B)', common_name='blue'),
    ],
)

MS_ASSET_CONFIG = AssetConfig(
    title="MS Orthomosaic Asset",
    nodata=-32767.0,
    data_type="Float32",
    bands=[
        BandConfig(name="b1", description="Green (G): 560 ± 16 nm", common_name="green", center_wavelength=0.56, full_width_half_max=0.016),
        BandConfig(name="b2", description="Red (R): 650 ± 16 nm", common_name="red", center_wavelength=0.65, full_width_half_max=0.016),
        BandConfig(name="b3", description="Red Edge (RE): 730 ± 16 nm", common_name="rededge", center_wavelength=0.73, full_width_half_max=0.016),
        BandConfig(name="b4", description="Near infrared (NIR): 860 ± 26 nm", common_name="nir", center_wavelength=0.86, full_width_half_max=0.026),
    ],
)

class ItemConfig(BaseModel):
    title: str
    description: str | None = None
    platform: str = 'dji-mavic-3m'
    instruments: list[str]
    asset_config: AssetConfig

RGB_ITEM_CONFIG = ItemConfig(
    title="RGB Orthomosaic Item",
    instruments=["dji-mavic-3m-rgb-camera"],
    asset_config=RGB_ASSET_CONFIG
)

MS_ITEM_CONFIG = ItemConfig(
    title="MS Orthomosaic Item",
    instruments=["dji-mavic-3m-ms-camera"],
    asset_config=MS_ASSET_CONFIG
)

In [173]:
from pystac.extensions.projection import ProjectionExtension

def create_bands_info(asset_path: Path, band_configs: list[BandConfig]):
    # Read in COG info
    cog_info = gdal.alg.raster.info(asset_path, stats=True, hist=True).Output()
    
    # Check number of bands is the same
    bands_info = cog_info['bands']

    combined_bands = zip(bands_info, band_configs)
    bands = []
    for cog_band, config_band in combined_bands:
        band = {
            "name": config_band.name,
            "description": config_band.description,
            "eo:common_name": config_band.common_name
        }

        # If statistics are there, add them to the band data
        statistics = {}
        if 'maximum' in cog_band:
            statistics["maximum"] = cog_band['maximum']
        if 'minimum' in cog_band:
            statistics["minimum"] = cog_band['minimum']
        if 'mean' in cog_band:
            statistics["mean"] = cog_band['mean']
        if 'stdDev' in cog_band:
            statistics["stddev"] = cog_band['stdDev']
        if statistics:
            band['statistics'] = statistics

        # If histogram info is there, add 
        if 'histogram' in cog_band:
            band['raster:histogram'] = cog_band['histogram']
            
        # If extra config is included add it
        if config_band.center_wavelength:
            band['eo:center_wavelength'] = config_band.center_wavelength
        if config_band.full_width_half_max:
            band["eo:full_width_half_max"] = config_band.full_width_half_max

        bands.append(band)
    
    return bands

def create_asset(asset_path: Path, asset_config: AssetConfig) -> pystac.Asset:
    # Read in COG info
    cog_info = gdal.alg.raster.info(asset_path).Output()
    
    # Check number of bands is the same
    bands_info = cog_info['bands']
    num_cog_bands = len(bands_info)
    num_config_bands = len(asset_config.bands)
    assert num_config_bands == num_cog_bands, f"Expected {num_config_bands} bands. COG had {num_cog_bands} bands."

    # Check nodata value is consistent across all bands
    nodata = bands_info[0]["noDataValue"]
    assert all(band["noDataValue"] == nodata for band in bands_info), "All bands must share the same nodata"
    # Check nodata value is the same as provided
    assert asset_config.nodata == nodata, f"Expected no data value of {asset_config.nodata}. Instead found {nodata}."

    # Change datatype from GDAL naming to STAC naming
    if asset_config.data_type == "Byte":
        data_type = "uint8"
    elif asset_config.data_type == "Float32":
        data_type = "float32"
    else:
        raise Exception(f"Unhandled data type {data_type}")

    
    asset = pystac.Asset(
        href=str(asset_path),
        title=asset_config.title,
        description=asset_config.description,
        roles=["data"],
        media_type=pystac.MediaType.COG,
        
        extra_fields={
            "data_type": data_type,
            "nodata": nodata,
            "bands": create_bands_info(asset_path, asset_config.bands),
            "raster:sampling": "area",
        },   
    )

    return asset

def create_item(asset_path: Path, item_config: ItemConfig):
    id = asset_path.name.replace('.cog.tif', '')
    name_parts = asset_path.name.split('_')
    
    item_date = datetime.strptime(name_parts[0], '%Y%m%d')
    item_date = item_date.replace(tzinfo=ZoneInfo('Australia/Hobart'), hour=12)
    item_date = item_date.astimezone(ZoneInfo('UTC'))

    site = name_parts[1]
    agl = int(name_parts[3].replace('mAGL', ''))

    cog_info = gdal.alg.raster.info(asset_path).Output()
    geometry = cog_info['wgs84Extent'] 
    bbox = pystac.utils.geometry_to_bbox(geometry)

    title = f"{site} {item_config.title}"

    item = pystac.Item(
        id=id,
        geometry=geometry,
        bbox=bbox,
        datetime=item_date,
        properties={
            # Common Metadata
            "title": title,
            "description": item_config.description if item_config.description else title,
            "mission": site,
            "platform": item_config.platform,
            "instruments": item_config.instruments,
            # Custom to NatureScan
            "site": site,
            "agl_m": agl,
        },
        assets={
            "main": create_asset(asset_path, item_config.asset_config)
        },
        # Add Raster and EO extensions 2.0.0 as I've done them manually
        stac_extensions=[
            "https://stac-extensions.github.io/raster/v2.0.0/schema.json",
            "https://stac-extensions.github.io/eo/v2.0.0/schema.json"
        ],
    )

    # Add Projection Extension
    proj = ProjectionExtension.ext(item, add_if_missing=True)
    stac_info = cog_info['stac']
    proj.epsg = stac_info['proj:epsg']
    proj.wkt2 = stac_info['proj:wkt2']
    proj.shape = stac_info['proj:shape']
    proj.projjson = stac_info['proj:projjson']

    # Ordering via: https://github.com/stac-extensions/projection?tab=readme-ov-file#projtransform
    g = cog_info['geoTransform']
    proj.transform = [  g[1], g[2], g[0],
                        g[4], g[5], g[3],
                        0,    0,    1]
    


    return item

create_item(cogs[1], RGB_ITEM_CONFIG)

<Item id=20241207_SANSSTP002_m3m_50mAGL_ortho_rgb>

In [ ]:
# Create the catalog
catalog = pystac.Catalog(
    id='naturescan', 
    description='TerraLuma NatureScan Catalog', 
    title='NatureScan Catalog',
    catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED
)
catalog.normalize_and_save(str(stac_out_dir), catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED)

terra_luma_provider = pystac.Provider(
    name="TerraLuma", 
    description="TerraLuma University of Tasmania",
    roles=[pystac.ProviderRole.HOST, pystac.ProviderRole.LICENSOR, pystac.ProviderRole.PRODUCER, pystac.ProviderRole.PROCESSOR],
    url='https://www.utas.edu.au/research/projects/terraluma'
)

rgb_cogs = input_dir.rglob('*rgb.cog.tif')
ms_cogs = input_dir.rglob('*ms.cog.tif')

rgb_items = [create_item(cog, RGB_ITEM_CONFIG) for cog in rgb_cogs]
ms_items = [create_item(cog, MS_ITEM_CONFIG) for cog in ms_cogs]

rgb_collection = pystac.Collection(
    id="naturescan-rgb",
    title='NatureScan RGB Orthomosaics',
    description='NatureScan RGB Orthomosaics',
    extent=pystac.Extent.from_items(rgb_items),
    providers=[terra_luma_provider],
)
rgb_collection.add_items(rgb_items)
catalog.add_child(rgb_collection, title='NatureScan RGB Orthomosaics')

ms_collection = pystac.Collection(
    id="naturescan-ms",
    title='NatureScan MS Orthomosaics',
    description='NatureScan MS Orthomosaics',
    extent=pystac.Extent.from_items(ms_items),
    providers=[terra_luma_provider],
)
ms_collection.add_items(ms_items)
catalog.add_child(ms_collection, title='NatureScan MS Orthomosaics')
catalog.describe()

* <Catalog id=naturescan>
    * <Collection id=naturescan-rgb>
      * <Item id=20241207_SANSSTP002_m3m_50mAGL_ortho_rgb>
      * <Item id=20241208_SANSSTP005_m3m_60mAGL_ortho_rgb>
      * <Item id=20241210_SANSSTP020_m3m_50mAGL_ortho_rgb>
      * <Item id=20241209_SANSSTP010_m3m_50mAGL_ortho_rgb>
      * <Item id=20240812_SANSSTP009_m3m_70mAGL_ortho_rgb>
      * <Item id=20241208_SANSSTP006_m3m_110mAGL_ortho_rgb>
      * <Item id=20241210_SANSSTP014_m3m_50mAGL_ortho_rgb>
      * <Item id=20250722_SAABHC0001_m3m_50mAGL_ortho_rgb>
      * <Item id=20250723_SAABHC0004_m3m_50mAGL_ortho_rgb>
      * <Item id=20250724_SAABHC0005_m3m_50mAGL_ortho_rgb>
      * <Item id=20250724_SAABHC0002_m3m_50mAGL_ortho_rgb>
    * <Collection id=naturescan-ms>
      * <Item id=20241207_SANSSTP002_m3m_50mAGL_ortho_ms>
      * <Item id=20241208_SANSSTP005_m3m_60mAGL_ortho_ms>
      * <Item id=20241210_SANSSTP020_m3m_50mAGL_ortho_ms>
      * <Item id=20241209_SANSSTP010_m3m_50mAGL_ortho_ms>
      * <Item id=20

In [189]:
catalog.normalize_and_save(str(stac_out_dir), catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED)